In [86]:
import numpy as np
from Session import Session
import pickle
import pandas as pd

In [101]:
def mrr( rank, label ):
    for i, item in enumerate(rank[:20]):
        if item == label:
            return 1.0/(i + 1)
    return 0

def recall( rank, label ):
    return int(label in rank[:20])
    
def pred2rank_result( pred ):
    top20_indices = np.argsort(pred)[-20:]
    top20_indices = np.flip(top20_indices, 0)
    rank_result = [ candidates[i] for i in top20_indices ]
    return rank_result


In [88]:
with open('../output/STAN_result.csv', 'r') as f:
    STAN_df = pd.read_csv(f)
with open('../output/SKNN_result.csv', 'r') as f:
    SKNN_df = pd.read_csv(f)
with open('../output/VSKNN_result.csv', 'r') as f:
    VSKNN_df = pd.read_csv(f)
with open( '../output/itemCF_result.csv', 'r') as f:
    itemCF_df = pd.read_csv(f)
with open('../output/NARM_100epochs.csv', 'r') as f:
    NARM_df = pd.read_csv(f)
    

In [96]:
session_set = set(STAN_df.session_id) & set(SKNN_df.session_id) & set(VSKNN_df.session_id) & set(itemCF_df.session_id) & set(NARM_df.session_id)
x_test = []
y_test = []
for i in session_set:
    x_test.append( session_ids[i] )
    y_test.append( labels[i] )

In [105]:
for res_df, name in zip([STAN_df, SKNN_df, VSKNN_df, itemCF_df, NARM_df], ['STAN', 'SKNN', 'VSKNN', 'itemCF', 'NARM']):
    global_mrr = 0
    global_recall = 0
    for x, y in zip(x_test, y_test):
        rank = res_df.loc[res_df.session_id == x, 'item_id'].values
        local_mrr = mrr( rank, y )
        local_recall = recall( rank, y )

        global_mrr += local_mrr
        global_recall += local_recall
    
    global_mrr /= len(x_test)
    global_recall /= len(x_test)
    print(name, global_mrr, global_recall)

STAN 0.24972042843405248 0.6303992821893226
SKNN 0.2494927602187688 0.630004486316734
VSKNN 0.2499221139145428 0.6308479138627187
itemCF 0.17820801074057285 0.4406639748766263
NARM 0.29614350255209815 0.6913772992373262


In [112]:
# ensemble
global_mrr = 0
global_recall = 0
for x, y in zip(x_test, y_test):
    score = {}
    for res_df, weight in zip( [STAN_df, itemCF_df, NARM_df], [2, 1, 3]):
        for i, item in enumerate(res_df.loc[res_df.session_id == x, 'item_id'].values.tolist()):
            if item not in score:
                score[item] = 0
            score[item] += weight / (np.log(i + 5) + 1)

    rank = [ k for k, _ in sorted(score.items(), key=lambda item: item[1], reverse=True)[:20] ]

    local_mrr = mrr( rank, y )
    local_recall = recall( rank, y )

    global_mrr += local_mrr
    global_recall += local_recall
global_mrr /= len(x_test)
global_recall /= len(x_test)
print("ensemble", global_mrr, global_recall)

ensemble 0.2553769364955047 0.6883086585912965
